In [0]:
%pip install mlflow
#%pip install "git+https://github.com/mlflow/mlflow.git@refs/pull/10420/merge" lxml==4.9.3 transformers==4.30.2 langchain==0.0.342 databricks-vectorsearch==0.21
%pip install --upgrade langchain
%pip install faiss-cpu
%pip install pydantic==1.10.9
%pip install chromadb
%pip install InstructorEmbedding
%pip install gradio

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Cloning https://github.com/mlflow/mlflow.git (to revision refs/pull/10420/merge) to /tmp/pip-req-build-f8uq4451
  Running command git clone --filter=blob:none --quiet https://github.com/mlflow/mlflow.git /tmp/pip-req-build-f8uq4451
  Running command git fetch -q https://github.com/mlflow/mlflow.git refs/pull/10420/merge
  Running command git checkout -q 559d5bdd44e0034f9f4bc46b4752aba06f0e7dc5
  Resolved https://github.com/mlflow/mlflow.gi

In [0]:
dbutils.library.restartPython()

In [0]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [0]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/Workspace/Users/shalu.tyagi@hdfcbank.com/DataforRAG_final1.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()

In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.text_splitter import CharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_chunks = text_splitter.split_documents(data)

In [0]:
text_chunks

[Document(page_content='Instruction: How can I add a nominee for my NRO Savings Account?', metadata={'source': '/Workspace/Users/shalu.tyagi@hdfcbank.com/DataforRAG_final.csv', 'row': 0}),
 Document(page_content='response: If you hold NRO Account in single holding relationship then you can add nominee by logging into netbanking > Request > Add / Modify nominee. If you hold NRO Account in joint holding relationship, then please share DA1 form signed by all holders.You may download the form from our website (Visit www.hdfcbank.com/nri > Click on Forms Center > NRI > Service Related Forms) and you may also avail of our Mailbox facility to send your respective form.', metadata={'source': '/Workspace/Users/shalu.tyagi@hdfcbank.com/DataforRAG_final.csv', 'row': 0}),
 Document(page_content='Instruction: What is a personal loan?\nresponse: A Personal Loan is a loan that can be availed for any personal purpose such as as home renovation, wedding, medical emergencies, travelling, debt repayments

In [0]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
from langchain.vectorstores import Chroma
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=text_chunks, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)


# persiste the db to disk
vectordb.persist()
vectordb = None

# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)



In [0]:
from langchain.llms import MlflowAIGateway
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import gradio as gr
from mlflow.gateway import set_gateway_uri, create_route, query,delete_route

set_gateway_uri("databricks")

mosaic_completion_route = MlflowAIGateway(
  gateway_uri="databricks",
  route="completion"
)

# Wrap the prompt and Gateway Route into a chain

template = """[INST] <<SYS>>
You are Banking Question Answering Machine. Answer Accordingly
<</SYS>>

{context}

{question} [/INST]

"""
prompt = PromptTemplate(input_variables=['context', 'question'], template=template)
retrieval_qa_chain = RetrievalQA.from_chain_type(llm=mosaic_completion_route, chain_type="stuff", retriever=vectordb.as_retriever(), chain_type_kwargs={"prompt": prompt})


def generating_text(prompt,route_param,token):
    # Create a Route for text completions with MosaicML Inference API 
    create_route(
        name=route_param,
        route_type="llm/v1/completions",
        model={
            "name": "llama2-70b-chat",
            "provider": "mosaicml",
            "mosaicml_config": {
            "mosaicml_api_key": "30HUkJ.UEvkMW9tAyMS7v91L4-e-gA0r.MKC0GUBphZ.2y5"
            }
        }
    )

    response1 = query(
    route=route_param,
    data={"prompt": prompt,"temperature": 0.1,
        "max_tokens": token}
    )

    return(response1)

import string
import random
 
# initializing size of string
N = 7

def greet(Input,chat_history):
    RouteName1="text1"
    RouteName2="text2"
    system="you are a translator which converts english to hindi. Please translate the given text to hindi language and only return the content translated. no explanation"
    system1="you are a translator which converts hindi to english. Please translate the given text to english language from hindi language and only return the content translated. no explanation"
    prompt=f"[INST] <<SYS>> {system1} <</SYS>> {Input}[/INST]"
    delete_route("text1")
    result=generating_text(prompt,RouteName1,400)
    res=result['candidates'][0]['text']
    t=retrieval_qa_chain.run(res)
    prompt2=f"[INST] <<SYS>> {system} <</SYS>> {t} [/INST]"
    delete_route("text2")
    token=800
    result1=generating_text(prompt2,RouteName2,token)
    chat_history.append((Input, result1['candidates'][0]['text']))
    return "", chat_history


with gr.Blocks(theme=gr.themes.Soft(primary_hue=gr.themes.colors.blue, secondary_hue=gr.themes.colors.red)) as demo:
    gr.Markdown("## सखा- भाषा अब कोई बाधा नहीं है")
    chatbot = gr.Chatbot(height=400) #just to fit the notebook
    msg = gr.Textbox(label="Prompt",placeholder="अपना प्रश्न हिंदी में यहां दर्ज करें",max_lines=2)
    with gr.Row():
        btn = gr.Button("Submit")
        clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
    # btn = gr.Button("Submit")
    # clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
    btn.click(greet, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(greet, inputs=[msg, chatbot], outputs=[msg, chatbot])
    gr.Examples([["एचडीएफसी बैंक का कस्टमर केयर नंबर क्या है?"],["गोल्ड लोन क्या है??"],['गोल्ड लोन के लिए आवश्यक दस्तावेज।']], inputs=[msg,chatbot])

gr.close_all()
demo.launch(share=True,debug=True)


<command-2901883420740584>:7: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/deprecation.html for more details.
  set_gateway_uri("databricks")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-54970c6c-e7bc-4876-815b-e41a9250e594/lib/python3.10/site-packages/langchain/llms/mlflow_ai_gateway.py:64: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/deprecation.html for more details.
  mlflow.gateway.set_gateway_uri(self.gateway_uri)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f561d3857f541419b7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<command-2901883420740584>:63: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/deprecation.html for more details.
  delete_route("text1")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-54970c6c-e7bc-4876-815b-e41a9250e594/lib/python3.10/site-packages/mlflow/gateway/fluent.py:129: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/deprecation.html for more details.
  MlflowGatewayClient().delete_route(name)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-54970c6c-e7bc-4876-815b-e41a9250e594/lib/python3.10/site-packages/mlflow/gateway/client.py:44: FutureWarning: MLflow AI gateway is deprecated and has been replaced by the deployments API for generative AI. See https://mlflow.org/docs/latest/llms/gateway/deprecation.html for more details.
  self._gateway_uri = gateway_uri or get_ga